In [1]:
import pandas as pd
file = pd.read_csv('textNtags.csv')
data = pd.DataFrame(file)
full_data = data

from sklearn.model_selection import train_test_split
data = full_data.sample(n = 5000, replace = False)
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['Tag'], test_size=0.10, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0.00009, max_features=200000, smooth_idf=True, norm="l2", 
tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,3))

x_train_multilabel = vectorizer.fit_transform(X_train)
x_test_multilabel = vectorizer.transform(X_test)

/home/mv/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# import dill
# dill.pickle(vectorizer,open('vectorizer.pkl','wb'))

In [ ]:
import sklearn.metrics as metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier

classifier = OneVsRestClassifier(SGDClassifier(loss='log_loss', alpha=0.00001, penalty='l1'), n_jobs=-1)
classifier.fit(x_train_multilabel, y_train)

predictions = classifier.predict(x_test_multilabel)
print("accuracy:", metrics.accuracy_score(y_test, predictions))
print("macro f1 score:", metrics.f1_score(y_test, predictions, average='macro'))
print("micro f1 score:", metrics.f1_score(y_test, predictions, average='micro'))
print("hamming loss:", metrics.hamming_loss(y_test, predictions))


/home/mv/.local/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/mv/.local/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/mv/.local/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/mv/.local/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} an

accuracy: 0.036
macro f1 score: 0.00605744012743054
micro f1 score: 0.036
hamming loss: 0.964


In [ ]:
pred = pd.DataFrame(predictions)
display(pred)
test = pd.DataFrame(X_test)
test = test.reset_index()
display(test)
result = test.join(pred)
display(result)

In [ ]:
# import pickle
# pickle.dump(classifier,open('model.pkl','wb'))

In [ ]:
def personal_predict(new_question):
    vector = vectorizer.transform([new_question])
    predicted_tags = classifier.predict(vector)
    return predicted_tags
    



new_question = """How to avoid instanceof and casting interface to class

I'd like to avoid instanceof and casting interface to class. I have method with parameter of interface FFF and depending on whether fff is class AAA or TTT I call appropriate private method. In one of these methods I have to call fff.methodSpecificForOnlyOneImplementationOfInterfaceFFF

I have entities:

public interface FFF {
    Long getShipperId();

    default void updatePermissions(final Permission updatedPermissions) {
        ...
    }
}

public class AAA implements FFF {
    
}

public class TTT implements FFF {
    
}
And I have Service:

public void updatePermissions(FFF fff) {
        final SomeClass someVariable;
        if (fff instanceof TTT) {
            someVariable = Optional.ofNullable(...)
                    .map(...)
                    .orElseGet(() -> createDefaultSettings(fff.getShipperId()));
        } else {
            someVariable= Optional.ofNullable(...)
                    .map(...)
                    .orElseGet(() -> returnPermissions(forwardable));
        }
        fff.updatePermissions(someVariable);
}

private createDefaultSettings(Long id){
...
}

private returnPermissions(FFF fff) {
    AAA aaa= (AAA) fff;
        return Optional.ofNullable(aaa.getWindow()) // There is no getWindow() in FFF interface
                .map(Window::getPermissions)
                .map(...)
                .orElseGet(() -> createDefaultSettings(aaa.getShipperId()));
}

How to avoid instanceof and casting AAA aaa= (AAA) fff;? Or maybe that's ok? Maybe there is better solution for it?
 """

# Generate tags for the new question
generated_tags = personal_predict(new_question)

# Print the generated tags
print("Generated tags:", generated_tags)

Generated tags: ["['c#']"]


: 